In [1]:
# %% Imports
import pandas as pd
# Makes sure we see all columns
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install interpret module
!pip install interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 70.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 14.3 MB/s eta 0:00:00
   ━━

In [4]:
from interpret.glassbox import (LogisticRegression, ClassificationTree)
from interpret import show
from sklearn.metrics import f1_score, accuracy_score

In [24]:
# Create DataLoader class and pre-process data
class DataLoader():
    def __init__(self):
        self.data = None

    def load_data(self, path='/content/drive/MyDrive/healthcare-dataset-stroke-data.csv'):
        self.data = pd.read_csv(path)

    def preprocess_data(self):
        categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
        encode = pd.get_dummies(self.data[categorical_cols], prefix=categorical_cols)

        self.data = pd.concat([encode, self.data], axis=1)

        self.data = self.data.drop(categorical_cols, axis=1)

        self.data.bmi = self.data.bmi.fillna(0)

        self.data.drop(['id'], axis=1, inplace=True)


        for col in self.data.select_dtypes(include=['bool']).columns:
            self.data[col] = self.data[col].astype(int)

    def get_data_split(self):
        X = self.data.iloc[:, :-1]
        y = self.data.iloc[:, -1]
        return train_test_split(X, y, test_size=0.2, random_state=2021)

    def Oversample(self, X_train, y_train):

        over_sampling = RandomOverSampler(sampling_strategy='not majority')
        x_np = X_train.to_numpy()
        y_np = y_train.to_numpy()
        x_np, y_np = over_sampling.fit_resample(x_np, y_np)


        x_np = pd.DataFrame(x_np, columns=X_train.columns)
        y_np = pd.Series(y_np, name=y_train.name)

        return x_np, y_np

In [25]:
# %% Load and preprocess data
data_loader = DataLoader()
data_loader.load_data()
data_loader.preprocess_data()

In [26]:
# Split the data for evaluation
X_train, X_test, y_train, y_test = data_loader.get_data_split()
print(X_train.shape, X_test.shape)
# Oversample the train data
X_train, y_train = data_loader.Oversample(X_train, y_train)
print("After oversampling: ", X_train.shape)

(4088, 21) (1022, 21)
After oversampling:  (7778, 21)


In [27]:
# %% Fit logistic regression model
lr = LogisticRegression(random_state=20, feature_names=X_train.columns, penalty='l1', solver='liblinear')
lr.fit(X_train, y_train)
print("Training finished")

Training finished


In [28]:
# %% Evaluate logistic regression model
y_pred = lr.predict(X_test)
print("F1 score: ", {f1_score(y_test, y_pred, average='macro')})
print("Accuracy: ", {accuracy_score(y_test, y_pred)})

F1 score:  {0.5151383678745968}
Accuracy:  {0.7367906066536204}


In [29]:
# %% Explain local prediction
lr_local = lr.explain_local(X_test[:100], y_test[:100], name='Logistic Regression')
show(lr_local)

In [30]:
# %% Fit decision tree model
tree = ClassificationTree()
tree.fit(X_train, y_train)
print("Training finished")
y_pred = tree.predict(X_test)
print("F1 score: ", {f1_score(y_test, y_pred, average='macro')})
print("Accuracy: ", {accuracy_score(y_test, y_pred)})


Training finished
F1 score:  {0.5259920468974536}
Accuracy:  {0.761252446183953}


In [32]:
# %% Explain local prediction
tree_local = tree.explain_local(X_test[:100], y_test[:100], name='Tree')
show(tree_local)